In [1]:
import librosa
import pandas as pd
from librosa.effects import pitch_shift
import os
import pathlib
import random
from sklearn.metrics import accuracy_score, balanced_accuracy_score, recall_score, precision_score, f1_score
import pickle
from sklearn.metrics import accuracy_score, balanced_accuracy_score, recall_score, precision_score
import datetime
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from librosa.feature import mfcc
import scipy.io.wavfile as wav
import tensorflow as tf
import time
import scipy.signal as signal
import keras
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.optimizers import adam_v2
from keras.layers import Input, Conv1D, SeparableConv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization,Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

# This notebook can be run sequentially without user input. Just run each block sequentially until tests begin.
![](evp_output.png)

# Read in Required CSVs/Models

In [6]:
#read in truth data for cellphone range from the sUAS and the flight directory of all flights
ranges_df = pd.read_csv("datasets\A3_Range_truth_dot256ms.csv")
parent_directory=os.path.abspath(os.path.join(os.getcwd(), ".."))
truthData=pd.read_csv(parent_directory+"\\flight_directories\\A3_flight_directory.csv")
mean_std = pd.read_csv("datasets\mean_std_training_ds_wo_A1_matrice_phantom.csv")
mean = np.array(mean_std.loc[:, 'mean'])
std = np.array(mean_std.loc[:, 'std'])
#### IF using ANN ####
json_file = open(
    r"models\ANN_LT80_wo_A1_A3_added.json",
    'r')
loaded_network_json = json_file.read()
json_file.close()
loaded_network = tf.keras.models.model_from_json(loaded_network_json)
loaded_network.load_weights(
    r"models\ANN_LT80_wo_A1_A3_added").expect_partial()

loaded_network.compile(optimizer='nadam',
                       loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                       metrics=['accuracy'], )
#### IF using SVM ####
filename = 'models/finalized_svm_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

# Declare Necessary Lists

In [3]:
phone_slots = ["Phone_1-1", "Phone_1-3", "Phone_11", "Phone_12", "Phone_15", "Phone_2-3", "Phone_2-4", "Phone_2-5",
               "Phone_27", "Phone_28", "Phone_29", "Phone_3-1", "Phone_3-2", "Phone_3-3", "Phone_3-4", "Phone_3-5",
               "Phone_30", "Phone_5-2", "Phone_5-3", "Phone_5-4", "Phone_6-2", "Phone_6-3", "Phone_6-4", "Phone_6-5",
               "Phone_7-1", "Phone_7-2", "Phone_8", "Phone_9"]
notable_col_names = ['Pass', 'Segment', 'Phone', 'Lat', 'Lon', 'Alt', 'Range']
droneDict = {  # One hot encoding for labels probs should do it like I did below?
    "Drone": [1, 0],
    "Noise": [0, 1]
}
bad_runs = ["A3R3P1", "A3R6P1", "A3R4P4", "A3R4P5", "A3R3P4", "A3R3P5"]  #aka used for training
tuning_runs = ["A3R5P3", "A3R4P2", "A3R6P3"]

# Define Necessary Functions

In [4]:
DRONE=1
NOISE=0

def findClosestPhone(phoneRanges):  #phoneList is a list of all N phone ranges
    """This function returns the closest cellphone's prediction and associated truth value.
    (not really used for evaluation, can be ignored)
    :param phoneList: A list of all N phone ranges
    :returns best_prediction: the prediction of the closest cellphone
    :returns truth_val: associated truth value"""
    closest_range = 10000
    best_prediction = 0
    for x in phoneRanges:
        phone_dist = x[0]
        if phone_dist < closest_range:
            best_prediction = x[1]
            closest_range = phone_dist

    if closest_range <= 80:
        truth_val = 1
    else:
        truth_val = 0
    return int(best_prediction), truth_val


def weighting(maxVals):
    """This function returns the softmax normalization of all cellphone's zeroth MFCC values.
    :param maxVals: An array of all cellphone's zeroth MFCC for a given 256ms frame
    :returns weights: Weighting of all sensors (between 0.0-1.0 and sums to 1.0)"""
    layer = tf.keras.layers.Softmax(axis=-1)
    weights = layer(maxVals)
    return weights


def MFCCCalc(audioData, Fs):
    """
    Converts decoded wav file to MFCC feature space
    :param audioData: Numpy array of decoded audio wav file
    :param Fs: Sampling rate (8KHz)
    :return MFCC 40 coefficients
    """
    #audioData=audioData.numpy()
    data = audioData.astype(float)
    #coefs = mfcc(data, sr=sampleRate, hop_length=2048)
    coefs = mfcc(y=data, hop_length=2048, n_fft=2048, n_mfcc=40, sr=Fs)
    return coefs




def create_test_dataset(test_files, testTime, testName):
    """
    Creates feature dataset and label dataset.
    @param test_files: EagerTensor of file paths.
    @return list of features (ds), list of labels corresponding to feature dataset:
    """
    featuresLL = []
    numPhones = 0
    pass_df = ranges_df.loc[ranges_df[str(notable_col_names.index('Pass'))] == testName]
    for x in test_files:
        test_audio, sampleRate = librosa.load(x, sr=8000)
        if len(test_audio) == (testTime) * sampleRate and min(
                np.asarray(test_audio)) != 0:  #ensure data actually has sound and recorded correctly
            numPhones += 1
            x = str(x)
            phoneName = x.split('\\')[7][:-4] #Parses name of phone. Must be changed based on data directory
            newFeats = MFCCCalc(test_audio.squeeze(), Fs=8000)
            phone_seg = pass_df.loc[pass_df[str(notable_col_names.index('Phone'))] == phoneName]
            if len(np.array(phone_seg.iloc[:, 7])) > 0:
                featuresLL.append([phoneName, newFeats.transpose(), np.array(phone_seg.iloc[:, 7])])
    return featuresLL


def maxValues(features):
    """Returns each cellphone's zeroth MFCC value in a list
    :param features: list of all cellphone's MFCC values
    :returns maxVals: A list of all zeroth MFCC values"""
    maxVals = []
    for x in features:
        maxVals.append(x[0])  #average power of signal
    return maxVals


# FSMs, very basic, predict sUAS if val>number
def zeroBitPrediction(prevPredictState, prediction):  #prediction need to be 1 (noise) or -1 (drone)
    if prediction == NOISE:
        predVal = 0
    else:
        predVal = 1
    return prevPredictState, predVal


def twoBitPrediction(prevPredictState, prediction):  #prediction need to be 1 (noise) or -1 (drone)
    if prediction == NOISE:
        predVal = -1
    else:
        predVal = 1
    if prevPredictState + predVal > 1:
        actualPrediction = 1
    else:
        actualPrediction = 0
    prevPredictState = prevPredictState + predVal
    if prevPredictState > 3:
        prevPredictState = 3
    elif prevPredictState < 0:
        prevPredictState = 0
    return prevPredictState, actualPrediction


def threeBitPrediction(prevPredictState, prediction):  #prediction need to be 1 (noise) or -1 (drone)
    if prediction == NOISE:
        predVal = -1
    else:
        predVal = 1
    if prevPredictState + predVal > 3:
        actualPrediction = 1
    else:
        actualPrediction = 0
    prevPredictState = prevPredictState + predVal
    if prevPredictState > 7:
        prevPredictState = 7
    elif prevPredictState < 0:
        prevPredictState = 0
    return prevPredictState, actualPrediction


def fourBitPrediction(prevPredictState, prediction):  #prediction need to be 1 (noise) or -1 (drone)
    if prediction == NOISE:
        predVal = -1
    else:
        predVal = 1
    if prevPredictState + predVal > 15:
        actualPrediction = 1
    else:
        actualPrediction = 0
    prevPredictState = prevPredictState + predVal
    if prevPredictState > 31:
        prevPredictState = 31
    elif prevPredictState < 0:
        prevPredictState = 0
    return prevPredictState, actualPrediction


def majorityVoteTuning(testFeats, fusionThresh, networkThresh, FSMVal):
    """Tunes the core of the EVP by determining the best fusion threshold, NN confidence threshold, and FSM values.
    :param Testfeats: List of all usable devices' name [0],MFCC values [1], and range from sUAS [2]
     :param fusionThresh: Value for voting threshold
     :param networkThresh: Value for NN confidence threshold
     :param FSMVal: FSM selection
     :returns predictedList: List of EVP predictions for the test flight
     :returns y_prediction: 2D list of all cellphones' predictions
     :returns truth_valueList: EVP truth value"""
    y_prediction = []
    maxVals = []
    for x in testFeats:
        features_list = x[1]
        features_list -= mean
        features_list /= std
        if len(x[2]) > 0:
            #features_list= np.reshape(features_list, (len(features_list), 40, 1))
            maxVals.append(maxValues(features_list))
            start_time = time.perf_counter()
            pred = loaded_network.predict(features_list)[:] > networkThresh
            #pred=loaded_model.predict(features_list)[:]
            end_time = time.perf_counter()
            time_per_prediction.append((end_time - start_time) / len(pred))
            pred.astype(int)
            #pred = model.predict(x.squeeze()) #for SVM
            y_prediction.append(pred)

    maxVals = np.asarray(maxVals).transpose()
    vote_weights = weighting(maxVals)
    predictedList = []
    truth_valueList = []
    prevState = FSMVal ** 2 / 2
    for i in range(len(testFeats[0][2])):  # i is feature frame
        predictedDrone = 0
        phone_ranges = []
        for j in range(len(y_prediction)):  # j is phone
            if y_prediction[j][i] == 1:
                #predictedDrone+= 1*maxValueRankArr[i][j]
                predictedDrone += 1 * float(vote_weights[i][j])
            if len(testFeats[j][2] > 0):
                phone_ranges.append([testFeats[j][2][i], y_prediction[j][i]])  #this is the range at the current frame
            else:
                phone_ranges.append([100000, y_prediction[j][i]])
        if predictedDrone > fusionThresh:
            predictedVal = 1
        else:
            predictedVal = 0
        if FSMVal == 0:
            prevState, prediction = zeroBitPrediction(prevState, predictedVal)
        if FSMVal == 1:
            prevState, prediction = twoBitPrediction(prevState, predictedVal)
        if FSMVal == 2:
            prevState, prediction = threeBitPrediction(prevState, predictedVal)
        if FSMVal == 3:
            prevState, prediction = fourBitPrediction(prevState, predictedVal)
        predictedList.append(prediction)
        best_indiv_predict, truth_value = findClosestPhone(phone_ranges)
        truth_valueList.append(truth_value)

    return predictedList, y_prediction, truth_valueList

# Evaluate Performance on Nine Test Scenarios

In [5]:
indiv_phone_f1_dict = {}
indiv_phone_recall_dict = {}
indiv_phone_prec_dict = {}
for x in phone_slots:
    indiv_phone_f1_dict[x] = []
    indiv_phone_prec_dict[x] = []
    indiv_phone_recall_dict[x] = []
def majorityVoteTuning(testFeats, fusionThresh, networkThresh, FSMVal):
    """Tunes the core of the EVP by determining the best fusion threshold, NN confidence threshold, and FSM values.
    :param Testfeats: List of all usable devices' name [0],MFCC values [1], and range from sUAS [2]
     :param fusionThresh: Value for voting threshold
     :param networkThresh: Value for NN confidence threshold
     :param FSMVal: FSM selection
     :returns predictedList: List of EVP predictions for the test flight
     :returns y_prediction: 2D list of all cellphones' predictions
     :returns truth_valueList: EVP truth value"""
    y_prediction = []
    maxVals = []
    for x in testFeats:
        features_list = x[1]
        features_list -= mean
        features_list /= std
        if len(x[2]) > 0:
            #features_list= np.reshape(features_list, (len(features_list), 40, 1))
            maxVals.append(maxValues(features_list))
            start_time = time.perf_counter()
            pred = loaded_network.predict(features_list)[:] > networkThresh
            #pred=loaded_model.predict(features_list)[:]
            end_time = time.perf_counter()
            time_per_prediction.append((end_time - start_time) / len(pred))
            pred.astype(int)
            #pred = model.predict(x.squeeze()) #for SVM
            y_prediction.append(pred)

    maxVals = np.asarray(maxVals).transpose()
    vote_weights = weighting(maxVals)
    predictedList = []
    truth_valueList = []
    prevState = FSMVal ** 2 / 2
    for i in range(len(testFeats[0][2])):  # i is feature frame
        predictedDrone = 0
        phone_ranges = []
        for j in range(len(y_prediction)):  # j is phone
            if y_prediction[j][i] == 1:
                #predictedDrone+= 1*maxValueRankArr[i][j]
                predictedDrone += 1 * float(vote_weights[i][j])
            if len(testFeats[j][2] > 0):
                phone_ranges.append([testFeats[j][2][i], y_prediction[j][i]])  #this is the range at the current frame
            else:
                phone_ranges.append([100000, y_prediction[j][i]])
        if predictedDrone > fusionThresh:
            predictedVal = 1
        else:
            predictedVal = 0
        if FSMVal == 0:
            prevState, prediction = zeroBitPrediction(prevState, predictedVal)
        if FSMVal == 1:
            prevState, prediction = twoBitPrediction(prevState, predictedVal)
        if FSMVal == 2:
            prevState, prediction = threeBitPrediction(prevState, predictedVal)
        if FSMVal == 3:
            prevState, prediction = fourBitPrediction(prevState, predictedVal)
        predictedList.append(prediction)
        best_indiv_predict, truth_value = findClosestPhone(phone_ranges)
        truth_valueList.append(truth_value)

    return predictedList, y_prediction, truth_valueList
start = 0
EVP_avg = []
outPerform = []
evp_recall = []
evp_precision = []
indiv_precisions = []
indiv_recalls = []
indiv_accs = []
indiv_f1 = []
time_per_prediction = []
evp_f1 = []

for x in range(18):
    truthScenario = truthData.iloc[x]
    pass_num = truthScenario[0]
    scenario = truthScenario[1]
    run_num = truthScenario[2]
    start = truthScenario[3]
    stop = truthScenario[4]
    drone_gps_file = truthScenario[5]
    start_time = datetime.datetime(2021, 8, int(start[6:8]), int(start[9:11]), int(start[11:13]),
                                   int(start[13:15])).timestamp()
    stop_time = datetime.datetime(2021, 8, int(stop[6:8]), int(stop[9:11]), int(stop[11:13]),
                                  int(stop[13:15])).timestamp()
    testTime = int(stop_time - start_time)
    full_name = (str(scenario) + 'R' + str(run_num) + 'P' + str(pass_num)).strip()
    print(full_name)
    if full_name not in tuning_runs and full_name not in bad_runs:
        dataset_path: str = "C:\\Users\\rclendening\\researchData\\EscapeCell_DataWav_V2\\A3\\" + full_name
        Testdata_dir = pathlib.Path(dataset_path)
        phones = tf.io.gfile.glob(str(Testdata_dir) + '/*')
        # convert to MFCC space
        featuresLL = create_test_dataset(phones, testTime, testName=full_name)
        predictedList, y_prediction, truth_valueList = majorityVoteTuning(featuresLL, fusionThresh=0.2,
                                                                          networkThresh=0.9, FSMVal=2)
        ####### Storing results  ##########
        num_frames = len(predictedList)  # Length of one scenario in frames
        tot_indiv_score = 0
        tot_recall_score = 0
        tot_precision_score = 0
        tot_detectable = np.zeros(num_frames)
        for z in range(len(y_prediction)):  # Iterate over the N cellphones
            if len(featuresLL[z][2] > 0):
                val = featuresLL[z][2] < 80 #  Count number of frames where cellphone should detect sUAS
                tot_detectable += val
                phone_name = featuresLL[z][0]
                indiv_phone_f1_dict[phone_name].append(f1_score(y_true=val, y_pred=y_prediction[z][0:len(val)]))
                indiv_phone_recall_dict[phone_name].append(recall_score(y_true=val, y_pred=y_prediction[z][0:len(val)]))
                indiv_phone_prec_dict[phone_name].append(
                    precision_score(y_true=val, y_pred=y_prediction[z][0:len(val)]))
                indiv_accs.append(accuracy_score(y_true=val, y_pred=y_prediction[z][0:len(val)]))
                indiv_recalls.append(recall_score(val, y_prediction[z][0:len(val)]))
                indiv_precisions.append(precision_score(val, y_prediction[z][0:len(val)]))
                indiv_f1.append(f1_score(y_true=val, y_pred=y_prediction[z][0:len(val)]))
        #################### Score Reporting ####################
        indiv_f1_avg = np.average(indiv_f1)
        print(full_name)
        print("Number of used phones:", len(y_prediction))
        fusion_acc = accuracy_score(y_true=truth_valueList, y_pred=predictedList)
        fusion_f1_score = f1_score(y_true=truth_valueList, y_pred=predictedList)
        EVP_avg.append(fusion_acc)
        print("Average Recall value:", recall_score(truth_valueList, predictedList))
        print("Average Precision value:", precision_score(truth_valueList, predictedList))
        print("Accuracy for drone present:", fusion_acc)
        print("Indiv Accuracy for drone present:", indiv_f1_avg)
        performanceGain = 100 * ((fusion_f1_score - indiv_f1_avg) / indiv_f1_avg)
        evp_f1.append(fusion_f1_score)
        outPerform.append(performanceGain)
        evp_recall.append(recall_score(truth_valueList, predictedList))
        evp_precision.append(precision_score(truth_valueList, predictedList))

print("Indiv Recall", np.average(indiv_recalls))
print("Indiv Precision", np.average(indiv_precisions))
print("Indiv F1", np.average(indiv_f1))
print("EVP Recall", np.average(evp_recall))
print(" EVP Precision", np.average(evp_precision))
print("EVP Accuracy", np.average(EVP_avg))
print("EVP F1", np.average(evp_f1))
print("Indiv Accuracy", np.average(indiv_accs))
print("Outperform", np.average(outPerform))

A3R3P1
A3R3P2
12/12 [==============================] - 0s 3ms/step
A3R3P2
Number of used phones: 16
Average Recall value: 0.9381443298969072
Average Precision value: 0.8053097345132744
Accuracy for drone present: 0.9261213720316622
Indiv Accuracy for drone present: 0.659699708257214
A3R3P3
12/12 [==============================] - 0s 2ms/step
A3R3P3
Number of used phones: 17
Average Recall value: 0.8469387755102041
Average Precision value: 0.8058252427184466
Accuracy for drone present: 0.9069148936170213
Indiv Accuracy for drone present: 0.6698599715653795
A3R3P4
A3R3P5
A3R4P1
12/12 [==============================] - 0s 3ms/step
A3R4P1
Number of used phones: 20
Average Recall value: 0.8350515463917526
Average Precision value: 0.8709677419354839
Accuracy for drone present: 0.9259259259259259
Indiv Accuracy for drone present: 0.615720072390955
A3R4P2
A3R4P3
12/12 [==============================] - 0s 2ms/step
A3R4P3
Number of used phones: 18
Average Recall value: 0.8865979381443299
Averag